# Project 3 - Ill-posedness and Tikhonov regularisation

You may have noticed, that the optimisation routine needs many thousand steps to converge. In this project we will investigate why this is the case.

Let's start by setting up our optimisation problem.

In [ ]:
from pyplasmaopt import *
nfp = 2
(coils, currents, expansion_axis, eta_bar) = get_24_coil_data(nfp=nfp, ppp=8, at_optimum=False)
currents = [1e5 * x for x in   [-2.271314992875459, -2.223774477156286, -2.091959078815509, -1.917569373937265, -2.115225147955706, -2.025410501731495]]
stellarator = CoilCollection(coils, currents, nfp, True)
iota_target = 0.103
coil_length_target = 4.398229715025710
magnetic_axis_length_target = 6.356206812106860
eta_bar = -2.25
obj = SimpleNearAxisQuasiSymmetryObjective(
        stellarator, expansion_axis, iota_target, eta_bar=eta_bar,
        coil_length_target=coil_length_target, magnetic_axis_length_target=magnetic_axis_length_target)


Since the code can only compute the objective value and its first derivative, we will have to use finite differences to approximate the Hessian matrix.
The $i$-th column of the Hessian $H$ can be approximated via
$$ H(:, i) \approx \frac{\nabla f(x+\epsilon e_i) - \nabla f(x-\epsilon e_i)}{2\epsilon}$$
where $e_i$ is the $i$-th unit vector and for some small $\epsilon$ (e.g. $\epsilon=10^{-5}$).

## Tasks

1) Write a function that approximates the Hessian at a given point $x$. By Schwarz's theorem, we know that the Hessian should be a symmetric matrix. Is this the case for the approximation that you computed? If not, how can we 'symmetrize' the Hessian approximation?

2) Run the optimisation for 2000 iterations and then compute the eigenvalues of the Hessian approximation at the minimum. Plot the eigenvalues. What does it mean for the objective when some eigenvalues are very small, and some are very large? What does this mean for the performance of gradient descent vs (quasi-)Newton methods?

3) A popular method of regularising ill-posed objectives, is the so called Tikhonov regularisation. Instead of solving 
$$ \min_x f(x) $$
one solves 
$$ \min_x f(x) + \frac\alpha2 \| x-x_0\|^2 $$
where $\alpha>0$ and $x_0$ is the initial guess. Implement a Tikhonov regularisation and rerun the optimisation for varying values of $\alpha$. Look at the obtained objective value, coils, and eigenvalues of the Hessian.
   

## Solutions